In [13]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/LongRange3D/')
from models_motif import LongRangeDNN
from utils import concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale
#from sklearn.preprocessing import MinMaxScaler, StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
contacts='PE'
path='/users/mtaranov/datasets_3d/dist_matched_'+contacts+'/'

X_train_node1 = path+'motifs/train_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/train_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_test_node1 = path+'motifs/test_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/test_'+contacts+'_out_node2_3_0.0001/mat.npy'
X_valid_node1 = path+'motifs/valid_'+contacts+'_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/valid_'+contacts+'_out_node2_3_0.0001/mat.npy'

y_train = get_labels(path+'y_train_thres_10.npy')
y_test = get_labels(path+'y_test_thres_10.npy')
y_valid = get_labels(path+'y_valid_thres_10.npy')

X_train=concat_motifs([X_train_node1, X_train_node2])
X_test=concat_motifs([X_test_node1, X_test_node2])
X_valid=concat_motifs([X_valid_node1, X_valid_node2])

In [15]:
X_test.shape

(1812, 1, 1920, 2)

In [16]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)

#X_train_normalized_subsampled, y_train_subsampled = subsample_data(X_train_normalized, y_train)
#X_valid_normalized_subsampled, y_valid_subsampled = subsample_data(X_valid_normalized, y_valid)
#X_test_normalized_subsampled, y_test_subsampled = subsample_data(X_test_normalized, y_test)

#X_train_scaled_subsampled, y_train_subsampled = subsample_data(X_train_scaled, y_train)
#X_valid_scaled_subsampled, y_valid_subsampled = subsample_data(X_valid_scaled, y_valid)
#X_test_scaled_subsampled, y_test_subsampled = subsample_data(X_test_scaled, y_test)


# from motifs

In [17]:
for i in range(10):
    dnn_normalized = LongRangeDNN(num_features=1920, use_deep_CNN=True)
    validation_data = (X_valid_normalized, y_valid)
    dnn_normalized.train(X_train_normalized, y_train, validation_data)
    print(dnn_normalized.test(X_test_normalized, y_test))
    print(dnn_normalized.test(X_train_normalized, y_train))
    print(dnn_normalized.test(X_valid_normalized, y_valid))

Training model...
Train on 4824 samples, validate on 2872 samples
Epoch 1/100
4750/4824 [============================>.] - ETA: 0s - loss: 3.2242Epoch 0: validation loss: 0.682
Balanced Accuracy: 59.68%	 auROC: 0.645	 auPRC: 0.625	 auPRG: 0.294
Recall at 5%|10%|20% FDR: 0.7%|0.8%|2.6%	 Num Positives: 1436	 Num Negatives: 1436	 

4824/4824 [==============================] - 18s - loss: 3.2004 - val_loss: 0.6816
Epoch 2/100
4750/4824 [============================>.] - ETA: 0s - loss: 1.4837Epoch 1: validation loss: 0.680
Balanced Accuracy: 59.44%	 auROC: 0.668	 auPRC: 0.645	 auPRG: 0.350
Recall at 5%|10%|20% FDR: 1.5%|1.5%|1.9%	 Num Positives: 1436	 Num Negatives: 1436	 

4824/4824 [==============================] - 26s - loss: 1.4831 - val_loss: 0.6803
Epoch 3/100
4750/4824 [============================>.] - ETA: 0s - loss: 1.4167Epoch 2: validation loss: 0.670
Balanced Accuracy: 60.34%	 auROC: 0.675	 auPRC: 0.649	 auPRG: 0.369
Recall at 5%|10%|20% FDR: 0.1%|0.1%|2.2%	 Num Positives: 14

MemoryError: ('Error allocating 36000000 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).', "you might consider using 'theano.shared(..., borrow=True)'")

In [ ]:
#save model and weights
dnn_normalized.save(contacts+'_dnn_CONV_motifs', '/users/mtaranov/LongRange3D/')

# DeepLift

In [ ]:
weights='/users/mtaranov/LongRange3D/weights/'+contacts+'_dnn_CONV_motifs.weights.h5'
json_model='/users/mtaranov/LongRange3D/models/'+contacts+'_dnn_CONV_motifs.arch.json'

In [ ]:
scores = dnn_normalized.deeplift(X_valid_normalized, weights, json_model, batch_size=128)

In [ ]:
np.save('/users/mtaranov/LongRange3D/deeplift_scores/'+contacts+'_scores_dnn_CONV_motifs.npy', scores)

In [ ]:
plt.plot(np.mean(scores[:, 0, :, 0], axis=0))

In [ ]:
plt.plot(np.mean(scores[:, 0, :, 1], axis=0))

In [ ]:
vmin = np.min(np.concatenate((scores[:, 0, :, 0], scores[:, 0, :, 1]), axis=1))
vmax = np.max(np.concatenate((scores[:, 0, :, 0], scores[:, 0, :, 1]), axis=1))

In [ ]:
#plt.imshow(scores[:, 0, :, 0], cmap='hot', interpolation='nearest')
plt.pcolor(scores[:, 0, :, 0], cmap='hot', vmin=vmin, vmax=vmax) # cmap='RdBu'
plt.xlabel('motifs/features')
plt.ylabel('DeepLift Score')
plt.title('Node 1')
plt.colorbar()

In [ ]:
#plt.imshow(scores[:, 0, :, 1], cmap='hot', interpolation='nearest')
plt.pcolor(scores[:, 0, :, 1], cmap='hot', vmin=vmin, vmax=vmax) 
plt.xlabel('motifs/features')
plt.ylabel('DeepLift Score')
plt.title('Node 2')
plt.colorbar()

# Print prediction

In [ ]:
pred_probs_test = dnn_normalized.predict(X_test_normalized)
#np.save("model_predictions/test_set_upperTriangle_noDiag_reads_with_distances_full_valid.npy", pred_probs_test)
pred_probs_valid = dnn_normalized.predict(X_valid_normalized)
#np.save("model_predictions/valid_set_upperTriangle_noDiag_reads_with_distances_full_valid.npy", pred_probs_valid)
pred_probs_train = dnn_normalized.predict(X_train_normalized)
#np.save("model_predictions/train_set_upperTriangle_noDiag_reads_with_distances_full_valid.npy", pred_probs_train)

In [ ]:
indx_train = np.load(path+'indx_train_thres_10.npy')
indx_test = np.load(path+'indx_test_thres_10.npy')
indx_valid = np.load(path+'indx_valid_thres_10.npy')

In [ ]:
labels_all=(np.concatenate((y_train, y_valid, y_test), axis=0)).astype(int)
preds_all=(np.concatenate((pred_probs_train, pred_probs_valid, pred_probs_test), axis=0))
indx_all=np.concatenate((indx_train, indx_valid, indx_test), axis=0)

labels_chr16 = labels_all[np.ravel(np.where(indx_all[:,2]==16))]
preds_chr16 = preds_all[np.ravel(np.where(indx_all[:,2]==16))]
indx_chr16 = indx_all[np.ravel(np.where(indx_all[:,2]==16))]

In [ ]:
indx_chr16.max()

In [ ]:
NumberOfNodes=2840
preds_chr16_2d, labels_chr16_2d = get_2D(indx_chr16, preds_chr16, labels_chr16, NumberOfNodes)
# preds_test_2d, labels_test_2d = get_2D(indx_test, preds_test, labels_test, NumberOfNodes)

In [ ]:
printMatrix(preds_chr16_2d, '', 1, 1, title='preds')
printMatrix(labels_chr16_2d, '', 1, 1, title='Captures')

In [ ]:
zoomIn_window = (250,350)
NumberOfNodes=2840
plot_prediction(change_scale(preds_chr16_2d)+change_scale(preds_chr16_2d).T, change_scale(labels_chr16_2d)+change_scale(labels_chr16_2d).T, zoomIn_window, )